# 대일감정원 데이터사이언스 스터디 25회
* California Housing Price Prediction
* 2024.02.15

# 0.목표

* 머신러닝 회귀 모델 연습

# 1.기본 라이브러리 불러오기

In [130]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 2. Data Load

* california housing price dataset
* 출처 : kaggle.com > playground > Playground Series - Season 3, Episode 1
* Regression with a Tabular California Housing Dataset
* https://www.kaggle.com/competitions/playground-series-s3e1

* data 다운 받는 곳
* https://www.kaggle.com/competitions/playground-series-s3e1/data

* 다운 받아서 압축을 풀면 세 개의 파일이 나옵니다.
* train.csv, test.csv, sample_submission.csv

* 지난주와 같이 train.csv 파일로 학습시키고 test.csv 파일로 예측을 진행합니다.
* 예측값을 sample_submission.csv 파일 형식으로 저장해서 캐글에 제출하면 됩니다.

In [131]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# 3. 데이터 분석

In [132]:
train.head()

,id,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500


In [133]:
#  데이터 설명 

# id : 데이터 고유번호
# MedInc : 특정 지역의 중간 소득 (Median Income)
# HouseAge : 특정 지역의 중간 주택 연도 (Housing Median Age)
# AveRooms : 평균 방 갯수 (Average Rooms)
# AveBedrms : 평균 침실 갯수 (Average Bedrooms)
# Population : 특정 지역의 인구 (Population)
# AveOccup : 평균 자가 비율 (Average Occupancy)
# Latitude : 해당 특정 지역의 위도 (Latitude)
# Longitude : 해당 특정 지역의 경도 (Longitude)

# MedHouseVal : 해당 지역의 주택 가격의 중간값 (Median House Value) -> target

In [134]:
# 8개의 피처를 이용하여, Median House Value를 예측하는 것이 목표입니다.

# 3.데이터 분석

In [135]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37137 entries, 0 to 37136
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           37137 non-null  int64  
 1   MedInc       37137 non-null  float64
 2   HouseAge     37137 non-null  float64
 3   AveRooms     37137 non-null  float64
 4   AveBedrms    37137 non-null  float64
 5   Population   37137 non-null  float64
 6   AveOccup     37137 non-null  float64
 7   Latitude     37137 non-null  float64
 8   Longitude    37137 non-null  float64
 9   MedHouseVal  37137 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 2.8 MB


In [136]:
# 37,137개
# 결측치도 없고, 모두 숫자형 데이터입니다.

In [137]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24759 entries, 0 to 24758
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          24759 non-null  int64  
 1   MedInc      24759 non-null  float64
 2   HouseAge    24759 non-null  float64
 3   AveRooms    24759 non-null  float64
 4   AveBedrms   24759 non-null  float64
 5   Population  24759 non-null  float64
 6   AveOccup    24759 non-null  float64
 7   Latitude    24759 non-null  float64
 8   Longitude   24759 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 1.7 MB


In [138]:
# 24,759개
# target인 MedHouseVal이 없습니다.

In [139]:
print(train.duplicated().sum())
print(test.duplicated().sum())


0
0


In [140]:
# 중복 데이터도 없습니다. (모든 칼럼값이 중복인 데이터가 없다는 뜻입니다.)

# 4. 데이터 전처리

In [141]:
# 결측치 처리
# 라벨 인코딩 
# 불필요함

# 이상치는 여기서는 무시함.

### 1) 데이터 분리

In [142]:
# 타겟과 피처 분리
y = train['MedHouseVal']
X = train.drop(['MedHouseVal'], axis=1)

In [143]:
from sklearn.model_selection import train_test_split

In [144]:
# train 데이터를 다시 train과 valid로 분리
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [145]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((29709, 9), (7428, 9), (29709,), (7428,))

### 2) scaling

In [146]:
from sklearn.preprocessing import StandardScaler

In [147]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

In [148]:
# 순서가 중요합니다. 
# X_train으로 fit을 하고, 그 scaler로 X_train과 X_valid를 transform 해줍니다.

# 2,3행은 합쳐서 한줄로 많이 씁니다.
# X_train_scaled = scaler.fit_transform(X_train)